## 대용량 json 파일 구조 확인하기

In [ ]:
%%cmd
head C:/Users/default.DESKTOP-FVT3076/Desktop/data/data.json
#안되니까 git bash로 확인
#한줄 데이터라 byte 단위로 json 구조 확인
head $ head data.json -c 1000

In [87]:
!jupyter notebook --generate-config

Writing default config to: C:\Users\default.DESKTOP-FVT3076\.jupyter\jupyter_notebook_config.py


## 데이터 셋 분할

In [68]:
crawl_range = range(0,45998)
# crawl_range = range(45998,91998)
# crawl_range = range(91998,137994)
# crawl_range = range(137994,183992)
# crawl_range = range(183992,229990)

# crawl_range = range(229990,275988)
# crawl_range = range(275988,321986)
# crawl_range = range(321986,367984)
# crawl_range = range(367984,413982)
# crawl_range = range(413982,459989)

In [ ]:
cd ./data

## 데이터 전처리

In [ ]:
#데이터가 너무 커서 블록단위로 메모리에 올리는 ijson 사용
import pandas as pd
import ijson

f = open(f"../data/data.json", encoding="UTF-8")
objects = ijson.items(f, 'item')
df = pd.DataFrame(objects)
# df = pd.read_csv('./data/성북구_음식점.csv', sep=',', encoding='CP949')

# df = df[['업소명', '소재지도로명', '업태명', '주된음식', '행정동명', '소재지전화번호']]
# df.columns = ['name', 'address', 'cate1', 'cate2', 'dong', 'phone']
# df = df.drop_duplicates(['name'], keep='first')
# # 네이버 지도 검색창에 [~동 @@식당]으로 검색해 정확도를 높여야 합니다. 검색어를 미리 설정해줍시다.
# df['cate_mix'] = df['cate1'] + df['cate2']

print(df)

In [5]:
df.replace('&amp;','',inplace=True)

In [21]:
df['google_keyword'] = df['area'] + df['name']
df = df.loc[~df['google_keyword'].isnull()]

In [17]:
for i in range(0,df.shape[0]):
    if df.iloc[i]['area'] is None or df.iloc[i,-2] != df.iloc[i,-2]: #nan 찾는 방법
        df.iloc[i,-2] = df.iloc[i]['name']

In [22]:
# df.at[90,'google_keyword'] = df.iloc[90]['name']
# df.at[90,'google_keyword']
df.shape

(443849, 14)

In [ ]:
# 중복 키워드 삭제
df = df.drop_duplicates(['google_keyword'], keep='first') # 4371 사라짐
print(df['google_keyword'].value_counts())
print(df.shape)

In [27]:
#필요한 칼럼
temp = df[['name','area', 'address','latitude','longitude','google_keyword']]

In [29]:
#구글맵용 가게정보 data 5등분 하기
import os
from datetime import datetime

start = 0
for i in range(1, 6):
    end = (df1.shape[0]//5) * i
    temp = df1[start:end]
    file = 'GoogleMap용_가게정보_part_'+str(i) +'.csv'
    temp.to_csv(file, encoding='utf-8')
    print(file)
    start = end


GoogleMap용_가게정보_part_1.csv
GoogleMap용_가게정보_part_2.csv
GoogleMap용_가게정보_part_3.csv
GoogleMap용_가게정보_part_4.csv
GoogleMap용_가게정보_part_5.csv


In [13]:
#scweet용 가게정보 data 5등분 하기
import os
from datetime import datetime

df = df.drop_duplicates(['name'], keep='first') # 4371 사라짐

start = 0
for i in range(1, 6):
    end = (df.shape[0]//5) * i
    df1 = df['name'][start:end]
    file = 'scweet용_가게정보_' + str(end) + '행까지.csv'
    df1.to_csv(file, encoding='utf-8')
    print(file)
    start = end


scweet용_가게정보_49024행까지.csv
scweet용_가게정보_98048행까지.csv
scweet용_가게정보_147072행까지.csv
scweet용_가게정보_196096행까지.csv
scweet용_가게정보_245120행까지.csv


## ipynb to py

In [ ]:
!pip install nbconvert

In [45]:
!jupyter nbconvert --to script naver_search_api.ipynb

[NbConvertApp] Converting notebook naver_search_api.ipynb to script
[NbConvertApp] Writing 7482 bytes to naver_search_api.py


## pandas list column

In [28]:
import pandas as pd

df = pd.read_csv('../outputs/BERT용 data/BERT_0~20000행_2021-10-03.csv', sep=',', encoding='utf-8')


In [ ]:
df = df.loc[~df['google_review_txt'].isnull()]
df

In [ ]:
from ast import literal_eval

df['google_review_txt'] = df['google_review_txt'].apply(literal_eval)
df['google_review_txt'].iloc[0]

## send to ElasticSearch

In [ ]:
pip install elasticsearch

In [41]:
from elasticsearch import Elasticsearch
import pandas as pd

df = pd.read_csv('../outputs/ES용 data/ES_0~20000행_2021-10-03.csv', sep=',', encoding='utf-8')

In [43]:
# df.loc[~df['google_stars'].isnull()]
df.replace('&amp;','',inplace=True)
df

In [ ]:
def setStore(i, store):
    
    es = Elasticsearch(
        hosts=[{'host':'3.34.51.207', 'port':'9200'}],
        http_auth=('elastic','msg!234'))
    es.index(index="msg", document=store)


for d in df.values:
    print(d)
    store = {
        "name" : d[1],
        "area" : d[2],
        "address" : d[3],
        "latitude" : d[4],
        "longitude" : d[5]
    }  
    
    setStore(d[0],store)